<div >
<img src = "../banner.jpg" />
</div>

<a target="_blank" href="https://colab.research.google.com/github/ignaciomsarmiento/BDML_202402/blob/main/Lecture06/Notebook_Lasso_Causal.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Aplication


-   To illustrate how it works let me use this experiment from the General Social Survey (GSS)

-   GSS conducts surveys regular surveys on Americans think feel about different issues

-   For decades, scholars studying Americans' support for social welfare spending have noted the special disdain that americans harbor for programs labeled "welfare"

-   This phenomenon became the subject of sustained experimental inquiry in the mid-1980s, when the GSS included a question-wording experiment in its national survey of adults.


-   Respondents in each survey were randomly assigned to one of two questions about public spending.

-   *"too much" money is spent on assistance to the Poor (treatment) or Welfare (control)*

-   Various explanations put forward: stereotypes associated with welfare recipients and poor people, particularly racial stereotypes, and to political orientations such as individualism and conservatism .


## Loading packages

Let's load the packages

In [ ]:
# install.packages("pacman") #run this line if you use Google Colab

In [ ]:
set.seed(201911) 

require("pacman")
p_load("tidyverse",# Data wrangling       
        "fBasics",     # Summary statistics 
        "corrplot",    # Correlations 
        "psych",       # Correlation p-values 
        "hdm"         # High-Dimensional Metrics
)


## Loading the data

We will be working with the `welfare` dataset, from "Modeling Heterogeneous Treatment Effects in Survey Experiments with Baysian Additive Regression Trees" (Green and Kern, 2012) ).

Next, we load in the raw data and perform some data cleaning.

In [ ]:
# R script for reading data from github repository, set path to where you have the tutorial files saved.
df_experiment<-readRDS(url("https://github.com/ignaciomsarmiento/datasets/raw/main/welfare.rds"))


In [ ]:
head(df_experiment)

## Cleaning the data

Let's do some minimal housekeeping. First, we will drop the columns that aren't outcomes, treatments or (pre-treatment) covariates, since we won't be using those.


In [ ]:
covariate_names<- c("hrs1", "partyid", "income", "rincome", "wrkstat", "wrkslf", "age", "polviews", "educ", "earnrs", "race", "marital", "sibs", "childs", "occ80", "prestg80", "indus80", "res16", "reg16", "mobile16", "family16", "parborn", "maeduc", "degree", "sex", "born", "hompop", "babies", "preteen", "teens", "adults")

In [ ]:
# Combine all names
all_variables_names <- c("Y", "W", covariate_names)
df <- df_experiment %>% select(all_of(all_variables_names))

Next, let's drop any row that has missing values in them.

In [ ]:
# Drop rows containing missing values
df <- df %>% drop_na()

Some of the methods below don't accept `factor` variables, so let's change their type to `numeric` here. 

In [ ]:
# Converting all columns to numerical and add row id
df <- data.frame(lapply(df, function(x) as.numeric(as.character(x))))

df <- df %>% mutate_if(is.character,as.numeric)
df <- df %>% rowid_to_column( "ID")

head(df)

---

## Descriptive statistics

It's often useful to begin data analysis by simply looking at simple summary statistics. We use the function `basicStats` from the package `fBasics` to calculate them.

In [ ]:
# Make a data.frame containing summary statistics of interest
summ_stats <- fBasics::basicStats(df)
summ_stats <- as.data.frame(t(summ_stats))
# Rename some of the columns for convenience
summ_stats <- summ_stats %>% select("Mean", "Stdev", "Minimum", "1. Quartile", "Median",  "3. Quartile", "Maximum")
summ_stats <- summ_stats %>% rename('Lower quartile'= '1. Quartile', 'Upper quartile' ='3. Quartile')

In [ ]:
summ_stats

Presenting pairwise correlations is easy with the `corrplot` function from the `corrplot` package. On the table below, if the (unadjusted) p-value for a pair of variables is less than 0.05, its square is not colored.

In [ ]:
# Note: if the plot looks too cramped, try increasing fig.width and fig.height in the line above
pairwise_pvalues <- psych::corr.test(df, df)$p
corrplot(cor(df),
        type="upper",
        tl.col="black",
        order="hclust",
        tl.cex=1,
        addgrid.col = "black",
        p.mat=pairwise_pvalues,
        sig.level=0.05,
        number.font=10,
        insig="blank")

# PART I: ATE y CATE

 *"too much" money is spent on assistance to the Poor (treatment) or Welfare (control)*

In [ ]:
tapply(df$Y,df$W,mean)

- 48% agree that too much money is spent on Welfare
- 11% agree that too much money is spent on assistance to the Poor

In [ ]:
difference_in_means <- function(dataset) {
  # Filter treatment / control observations, pulls outcome variable as a vector
  y1 <- dataset %>% dplyr::filter(W == 1) %>% dplyr::pull(Y) # Outcome in treatment grp
  y0 <- dataset %>% dplyr::filter(W == 0) %>% dplyr::pull(Y) # Outcome in control group
  
  n1 <- sum(df[,"W"])     # Number of obs in treatment
  n0 <- sum(1 - df[,"W"]) # Number of obs in control
  
  # Difference in means is ATE
  tauhat <- mean(y1) - mean(y0)
  
  # 95% Confidence intervals
  se_hat <- sqrt( var(y0)/(n0-1) + var(y1)/(n1-1) )
  lower_ci <- tauhat - 1.96 * se_hat
  upper_ci <- tauhat + 1.96 * se_hat
  
  return(c(ATE = tauhat, lower_ci = lower_ci, upper_ci = upper_ci))
}

tauhat_rct <- difference_in_means(df)
tauhat_rct

In [ ]:
ols_ate <- lm(Y~W, data=df)

In [ ]:
fmla <- as.formula(paste("Y ~ W +", paste(covariate_names, collapse= "+")))
ols_cate <- lm(fmla, data=df)
stargazer::stargazer(ols_ate,ols_cate,type="text",digits = 5)

Second, we estimate the effect by the partialling out by Post-Lasso: 


In [ ]:
Y <- df$Y
W <- df$W

In [ ]:
X <- as.matrix(df[covariate_names])
head(X)

Third, we estimate the effect by the double selection method. Algorithmically the method is as follows:

- (1) Select controls $X_{ij}$’s that predict $y_i$ by Lasso.
- (2) Select controls $X_{ij}$’s that predict $W_i$ by Lasso.
- (3) Run OLS of yi on di and the union of controls selected in steps 1 and 2.


In [ ]:
doublesel.effect <- rlassoEffect(x=X, y=Y, d=W, method="double selection")
summary(doublesel.effect)

In [ ]:
data.frame(doublesel.effect$selection.index)

----